# Password Bypass (Setup Script)

**This is part of the general setup. The main exercise is called Exercise - Password Bypass - Exercise.ipynb**

First you'll need to select which hardware setup you have. You'll need to select both a `SCOPETYPE` and a `PLATFORM`. `SCOPETYPE` can either be `'OPENADC'` for the CWLite/CW1200 or `'CWNANO'` for the CWNano. `PLATFORM` is the target device, with `'CWLITEARM'`/`'CW308_STM32F3'` being the best supported option, followed by `'CWLITEXMEGA'`/`'CW308_XMEGA'`, then by `'CWNANO'`. As of CW 5.4, you can select the SimpleSerial version
used. For example:

```python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_1_1'
```

In [ ]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_1_1'

This code will connect the scope and do some basic setup. We're now just going to use a special setup script to do this. This script contains the commands we ran seperately before.

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"
import os

The following code will build the firmware for the target.

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/uebung_01
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2
ls guess_pw-CWNANO.hex

Finally, all that's left is to program the device, which can be done with the following line:

In [ ]:
cw.program_target(scope, prog, os.path.expanduser("../../hardware/victims/firmware/uebung_01/guess_pw-{}.hex".format(PLATFORM)))

To make interacting with the hardware easier, let's define a function to attempt a password and return a power trace:

In [ ]:
def cap_pass_trace(pass_guess):
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess+"\n")
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

We also don't need all of the default 5000 samples in the trace. 3000 is a good starting point for most targets:

In [ ]:
scope.adc.samples = 3000